In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import smacof

df = pd.read_csv('EFIplus_medit.zip',compression='zip', sep=";")

In [2]:
df.head()

,Site_code,Latitude,Longitude,Country,Catchment_name,Galiza,Subsample,Calib_EFI_Medit,Calib_connect,Calib_hydrol,...,Squalius malacitanus,Squalius pyrenaicus,Squalius torgalensis,Thymallus thymallus,Tinca tinca,Zingel asper,Squalius sp,Barbatula sp,Phoxinus sp,Iberochondrostoma_sp
0,ES_01_0002,38.102003,-4.096070,Spain,Guadalquivir,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,ES_02_0001,40.530188,-1.887796,Spain,Tejo,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,ES_02_0002,40.595432,-1.928079,Spain,Tejo,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,ES_02_0003,40.656184,-1.989831,Spain,Tejo,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,ES_02_0004,40.676402,-2.036274,Spain,Tejo,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


Let's check multicolinearity in the dataset

In [6]:
frame_scaled = StandardScaler.fit_transform(df, X=MatrixLike)
df_scaled = pd.DataFrame(data=frame_scaled, 
                                columns=df.columns)
df_scaled.head()

NameError: name 'MatrixLike' is not defined

We are now ready to run PCA.

### 1.3 Select the number of Principal Components (PC)

To select the number of PC, usually a Scree plot is produced to visualize the percentage of explained variance or the eigenvalues per component. Either the `elbow rule` - retaining all components before the point where the curve flattens out - or the `Kaiser's rule` - retaining components whose eigenvalues are greater than 1.

First we run PCA for 11 components (number of original variables).

In [5]:
pca = PCA(n_components=11)
pca.fit_transform(df_scaled)

NameError: name 'df_scaled' is not defined

We have run the PCA and now we can extract the proportion of explained variance and the eigenvalues as follows:

In [ ]:
eigenvalues = pca.explained_variance_ # eigenvalues
prop_var = pca.explained_variance_ratio_ # proportion of explained variance

Plot the scree plot with proportion of variance

In [ ]:
PC_numbers = np.arange(pca.n_components_) + 1
 
plt.plot(PC_numbers, 
         prop_var,
         'ro-')
plt.title('Scree Plot', fontsize=20)
plt.ylabel('Proportion of Variance', fontsize=30)
plt.show()

The `Elbow rule` is not easy to apply to our data.
We can try to apply the `Kaiser's rule` and for that we need to plot the eigenvalues instead.

In [ ]:
PC_numbers = np.arange(pca.n_components_) + 1
 
plt.plot(PC_numbers, 
         eigenvalues,
         'ro-')
plt.title('Scree Plot', fontsize=20)
plt.ylabel('Eigenvalues', fontsize=16)
plt.axhline(y=1, color='r', 
            linestyle='--')

According to this rule, 4 components should be retained. However, just for the purpose of ilustrating how to produce a PCA biplot vlisualization, we will retain only two components.

### 1.4 Visualize and interpret the PCA biplot

In [ ]:
pca = PCA(n_components=2)
PC = pca.fit_transform(df_scaled)

In [ ]:
pca_wine = pd.DataFrame(data = PC, 
                            columns = ['PC1', 'PC2'])
pca_wine.head(15)

In [ ]:
# "score" - Scores of each object (for each PC) - PC
# "coef" - PCA variable loadings (for each PC) - pca.components_
# labels - name of variables - list(df.columns)


def biplot(score,coef,labels=None): 
 
    xs = score[:,0] # PC1 object scores
    ys = score[:,1] # PC2 object scores 
    n = coef.shape[0] # number of dimensions (2)
    scalex = 1.0/(xs.max() - xs.min()) # to rescale scores
    scaley = 1.0/(ys.max() - ys.min()) # to rescale scores
    plt.scatter(xs * scalex,ys * scaley,
                s=6, 
                color='blue') # scatter plot using rescaled object scores
 
    for i in range(n):
        plt.arrow(0, 0, coef[i,0], 
                  coef[i,1],color = 'red',
                  head_width=0.01,
                  alpha = 0.5) # plot arrows for each variable
        plt.text(coef[i,0]* 1.15, 
                 coef[i,1] * 1.15, 
                 labels[i], 
                 color = 'red', 
                 ha = 'center', 
                 va = 'center') # variable labels for each arrow
 
    plt.xlabel("PC{}".format(1))
    plt.ylabel("PC{}".format(2))    
 
 
    plt.figure()

In [ ]:
plt.figure(figsize=(10, 8))
plt.title('Biplot of PCA')
 
biplot(PC, 
       np.transpose(pca.components_), 
       list(df.columns))

In [ ]:
# same biplot but with colors expressing wine quality

PC1 = pca_wine['PC1']/(pca_wine['PC1'].max() - pca_wine['PC1'].min())
PC2 = pca_wine['PC2']/(pca_wine['PC2'].max() - pca_wine['PC2'].min())

plt.figure(figsize=(10, 8))
plt.title('Biplot of PCA')
sns.scatterplot(x=PC1,
              y=PC2,
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )

n = np.transpose(pca.components_).shape[0] # number of dimensions (2)
for i in range(n):
        plt.arrow(0, 0, np.transpose(pca.components_)[i,0], 
                  np.transpose(pca.components_)[i,1], 
                  color = (0.1, 0.1, 0.1, 0.8),
                  head_width=0.02) # plot arrows for each variable
        plt.text(np.transpose(pca.components_)[i,0]* 1.15, 
                 np.transpose(pca.components_)[i,1] * 1.15, 
                 list(df.columns)[i], 
                 color = (0.1, 0.1, 0.1, 0.8), 
                 ha = 'center', 
                 va = 'center') # variable labels for each arrow
plt.legend(title='Wine quality')
plt.xlabel("PC{}".format(1))
plt.ylabel("PC{}".format(2))


### 1.5 Running PCA from scratch

The code that follows computes the same PCA but without depending on any specific PCA function.

NOTE: The first step - variable standardization - was already performed

Step 2: compute the covariance matrix 

In [ ]:
def covariance(x): 
    return (x.T @ x)/(x.shape[0]-1)

cov_mat = covariance(df_scaled) # np.cov(X_std.T)

Step 3: Compute the eigenvectors and eigenvalues of the covariance matrix

In [ ]:
from numpy.linalg import eig

# Eigendecomposition of covariance matrix
eig_vals, eig_vecs = eig(cov_mat)

# Adjusting the eigenvectors (loadings) that are largest in absolute value to be positive
max_abs_idx = np.argmax(np.abs(eig_vecs), axis=0)
signs = np.sign(eig_vecs[max_abs_idx, range(eig_vecs.shape[0])])
eig_vecs = eig_vecs*signs[np.newaxis,:]
eig_vecs = eig_vecs.T

print('Eigenvalues \n', eig_vals)
print('Eigenvectors \n', eig_vecs)

Step 4: Rearrange the eigenvectors and eigenvalues

In [ ]:
# Create a list of eigenvalue and eigenvector tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[i,:]) for i in range(len(eig_vals))]

# Sort the tuples from the highest to the lowest eigenvalues
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# For further usage
eig_vals_sorted = np.array([x[0] for x in eig_pairs])
eig_vecs_sorted = np.array([x[1] for x in eig_pairs])


Step 5: Select the number of PCs

In [ ]:
# Select top k eigenvectors
k = 2
W = eig_vecs_sorted[:k, :] # Projection matrix

eig_vals_total = sum(eig_vals)
explained_variance = [(i / eig_vals_total)*100 for i in eig_vals_sorted]
explained_variance = np.round(explained_variance, 2)
cum_explained_variance = np.cumsum(explained_variance)

print('Explained variance: {}'.format(explained_variance))
print('Cumulative explained variance: {}'.format(cum_explained_variance))

plt.plot(np.arange(1, 11 + 1), cum_explained_variance, '-o')
plt.xticks(np.arange(1,11+1))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance');
plt.show()

Step 6: Project the data with a biplot

In [ ]:
# 
X_proj = df_scaled.dot(W.T)


In [ ]:
# Create biplot
PC1 = X_proj.iloc[:, 0]/(X_proj.iloc[:, 0].max() - X_proj.iloc[:, 0].min())
PC2 =X_proj.iloc[:, 1]/(X_proj.iloc[:, 1].max() - X_proj.iloc[:, 1].min())

plt.figure(figsize=(10, 8))
plt.title('2 components, captures {:.1f} of total variation'.format(cum_explained_variance[1]))

sns.scatterplot(x=PC1,
              y=PC2,
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )

n = np.transpose(pca.components_).shape[0] # number of dimensions (2)
for i in range(n):
        plt.arrow(0, 0, np.transpose(pca.components_)[i,0], 
                  np.transpose(pca.components_)[i,1], 
                  color = (0.1, 0.1, 0.1, 0.8),
                  head_width=0.02) # plot arrows for each variable
        plt.text(np.transpose(pca.components_)[i,0]* 1.15, 
                 np.transpose(pca.components_)[i,1] * 1.15, 
                 list(df.columns)[i], 
                 color = (0.1, 0.1, 0.1, 0.8), 
                 ha = 'center', 
                 va = 'center') # variable labels for each arrow
plt.legend(title='Wine quality')
plt.xlabel("PC{}".format(1))
plt.ylabel("PC{}".format(2))

plt.xlabel('PC1'); plt.xticks([])
plt.ylabel('PC2'); plt.yticks([])
plt.show()

### 1. Multidimensional Scaling (MDS) and Non-Metric Multidimensional Scaling

The aim of this ordination method is to project objects from a higher-dimensional space to a lower-dimensional space while preserving the relative distances between those points as much as possible. PCoA (also known as MDS) can be viewed as a generalization of PCA that allows a much wider range of dissimilarity measures to be used (dissimilarities among objects in PCA are euclidean distances).

MDS and NMDS are implemented in the `mds()` function of `sklearn.manifold` module.

#### 1.1 Import modules and functions

Import necessary modules and functions:

In [ ]:
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

#### 1.2 Run MDS

We will run MDS using the same wine dataset used in PCA. We will use the standardized data (df_scaled).

In [ ]:
# by default the 'mds' argument is set to 'True', which means it will run a MDS
# Euclidean distances are also the default 
# Only two axis are extracted by default
mds = MDS(random_state=0, normalized_stress = False) 
mds_transf = mds.fit_transform(df_scaled)

Plot the result

In [ ]:
sns.scatterplot(x=mds_transf[:,0],
              y=mds_transf[:,1],
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )

We may use other distances to compute MDS. For that we need to run the function using a dissimilarity matrix as the input.
Let's try running MDS with Manhattan dissimilarities.

In [ ]:
dist_manhattan = manhattan_distances(df_scaled)
mds = MDS(dissimilarity='precomputed', random_state=0, normalized_stress = False)
mds_transf2 = mds.fit_transform(dist_manhattan)

sns.scatterplot(x=mds_transf2[:,0],
              y=mds_transf2[:,1],
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )

#### 1.3 Run NMDS

Non-Metric Multidimensional Scaling solved many issues of MDS, namely the computation of the meaured used to evaluate how much the distances in the MDS are related with the original dissimilarities. This methods is based on ranks of distances among objects, so the aim is to retain the relative position of objects from each other, not their distances. It is based on a iterative trial & error algorithm. 

In [ ]:
nmds = MDS(n_components=5, random_state=0, metric = False, normalized_stress="auto") # 5 components extracted so that stress is > 0.2
nmds_transf = nmds.fit_transform(df_scaled)

The match between object distance in MDS and the original dissimilarities is given by the stress, which is proportional to the residuals of that relationship. The stress of an MDS with a perfect match between MDS distances and original dissimilarities will be closer to zero. 

As a rule of thumb, an NMDS ordination with a stress value around or above 0.2 is deemed suspect and a stress value approaching 0.3 indicates that the ordination is arbitrary. Stress values equal to or below 0.1 are considered fair, while values equal to or below 0.05 indicate good fit.

In [ ]:
stress = nmds.stress_
print(stress)

In [ ]:
sns.scatterplot(x=nmds_transf[:,0],
              y=nmds_transf[:,1],
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )

It is also possible to use the method with dissimilarities other than Euclidean distances.

In [ ]:
# NMDS based on Manhattan distances
nmds2 = MDS(metric = False, random_state=0, normalized_stress = 'auto', dissimilarity='precomputed')
nmds_transf2 = nmds2.fit_transform(dist_manhattan)

sns.scatterplot(x=nmds_transf2[:,0],
              y=nmds_transf2[:,1],
              hue = df_wine['quality'].tolist(),
              linewidth=0,
              )

In [ ]:
stress = nmds2.stress_
print(stress)